In [350]:
import pandas as pd
import json
import requests
from python_secrets import *

In [351]:
df_main = pd.read_csv('incident2.csv')

In [352]:
tx_hash_list = [x for x in df_main[' user_tx'].to_list() if pd.notnull(x)]

# Infuria

In [353]:
url = f"https://mainnet.infura.io/v3/{infura_api_key}"

payload = json.dumps({
  "jsonrpc": "2.0",
  "method": "eth_getTransactionByHash",
  "params": [tx_hash_list[0]],
  "id": 1
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)
dct = response.json()['result']
dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
df_infuria = pd.DataFrame(dct, index=[0])

for tx_hash in tx_hash_list[1:]:
  payload = json.dumps({
    "jsonrpc": "2.0",
    "method": "eth_getTransactionByHash",
    "params": [tx_hash],
    "id": 1
  })
  headers = {
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  if response.ok:
    dct = response.json()['result']
    dct = {k: None if not v else v for k, v in dct.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct, index=[0])
    df_infuria = pd.concat([df_infuria, df_temp])
    
  else: 
    print(f"error code {response.status_code} for transaction {tx_hash}")


In [354]:
#merge infuria to main
df_main = df_main.merge(df_infuria, left_on = ' user_tx', right_on = 'hash', how ='outer')

In [355]:
df_main.columns

Index(['block_number', ' user_tx', ' fees', 'accessList', 'blockHash',
       'blockNumber', 'chainId', 'from', 'gas', 'gasPrice', 'hash', 'input',
       'maxFeePerGas', 'maxPriorityFeePerGas', 'nonce', 'r', 's', 'to',
       'transactionIndex', 'type', 'v', 'value', 'yParity'],
      dtype='object')

# Etherscan 

If we always have the dollar values from the tenderly api, we actually don't need to call ethersacan for the time stamps

In [356]:
block_number_list = list(set([x for x in df_main['block_number'].to_list() if pd.notnull(x)]))

In [357]:
url_eth = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number_list[0]}&apikey={eth_scan_api_key}"

response_eth = requests.request("POST", url_eth)

dct_eth = response_eth.json()['result']
dct_eth = {k: None if not v else v for k, v in dct_eth.items()} # making sure none of the values are empty
df_eth = pd.DataFrame(dct_eth, index=[0])

for block in block_number_list[1:]:
  url_temp = f"https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block}&apikey={eth_scan_api_key}"
  response_temp = requests.request("POST", url_temp)

  if response_temp.ok:
    dct_temp = response_temp.json()['result']
    dct_temp = {k: None if not v else v for k, v in dct_temp.items()} # making sure none of the values are empty
    df_temp = pd.DataFrame(dct_temp, index=[0])
    df_eth = pd.concat([df_eth, df_temp])
    
  else: 
    print(f"error code {response_temp.status_code} for block {block}")

KeyboardInterrupt: 

In [ ]:
df_eth['blockNumber'] = df_eth['blockNumber'].astype(int)

In [ ]:
#merge eth to main
df_main = df_main.merge(df_eth, left_on = 'block_number', right_on = 'blockNumber', how ='outer')

# Tenderly

In [365]:
headers = {
    'X-Access-Key': f'{tenderly_access_token}',
    'content-type': 'application/json',
}

columns = ['tx_hash', 'index', 'type', 'raw_amount', 'dollar_value', 'token_contract_address', 'token_name', 'token_dollar_value']
df_results = pd.DataFrame(columns = columns)
tx_hash_problem_list = []

for index, row in df_main.iterrows():
    tx_index_list = [0]
    tx_index_list.append(int(row['transactionIndex'], 0))
    for tx_index in tx_index_list:
        json_data = {
        'network_id': int(row['chainId'], 0),
        'from': row['from'],
        'to': row['to'],
        'input': row['input'],
        'block_number': row['block_number'],
        'transaction_index': tx_index,
        'simulation_type': 'quick'
    }
        response = requests.post(
        'https://api.tenderly.co/api/v1/account/aurelie/project/cowswap/simulate',
        headers=headers,
        json=json_data,
        )
        
        try:
            for data in response.json()['transaction']['transaction_info']['asset_changes']:
                tx_type = data['type']
                tx_raw_amount = data['raw_amount']
                tx_dollar_value = data['dollar_value']

                try:
                    contract_address = data['token_info']['contract_address']
                except:
                    contract_address = 'None'

                try:
                    token_name = data['token_info']['name']
                except:
                    token_name = 'None'
                try:
                    token_dollar_value = data['token_info']['dollar_value']
                except:
                    token_dollar_value = 'None'

                new_row = {
                    'tx_hash' : row['hash'],
                    'index' : tx_index, 
                    'type': tx_type, 
                    'raw_amount': tx_raw_amount, 
                    'dollar_value' : tx_dollar_value, 
                    'token_contract_address': contract_address, 
                    'token_name': token_name, 
                    'token_dollar_value': token_dollar_value
                    }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
        except:
            tx_hash_problem = df_main[df_main['input'] == row['input']]['hash']
            tx_hash_problem_list.append(tx_hash_problem)
            


In [367]:
grouped = df_results.groupby('tx_hash')
df_results_good = grouped.filter(lambda x: x['index'].nunique() == 2)

In [368]:
# number of transactions for which this approach worked
df_results_good.tx_hash.nunique()

16

In [373]:
df_results_good = df_results_good.groupby(['tx_hash', 'index'], as_index=False).first()
df_results_good['dollar_value'] = df_results_good['dollar_value'].astype(float)
df_results_good['asset_diff'] = df_results_good.groupby(['tx_hash'])['dollar_value'].diff()

In [374]:
#total potential loss in dollars
df_results_good.asset_diff.sum()

-1085.2218972332394

In [375]:
df_results_good = df_results_good.dropna(subset=['asset_diff'])[['tx_hash', 'asset_diff']]
df_results_good.to_csv('preliminary_results.csv')